# Proyecto de análisis de sentimientos con Python


Curso 2022/2023:
    Juan López Quirós
    Jose Ignacio Castro Vázquez

 Lo primero que hay que hacer es escoger o recopilar una primera versión de los datos necesarios que utilizarmos 
 para entrenar a nuestros modelos de Aprendizaje automático. 
 Tras haber estudiado los varios problemas con la API de twitter y las alternativas propuestas, nos decidimos por 
 buscar un dataset ya recopilado de tweets reales de la página web kaggle. En concreto nos decidimos por un dataset
 ya enfocado al análisis de sentimientos con más de 1.6 millones de tweets recopilados directamente de la API de twitter
 por lo que se ajusta perfectamente al proyecto.

 url : https://www.kaggle.com/datasets/kazanova/sentiment140

 Luego, tenemos que limpiar el dataset escogido para este proyecto.
 A nosotros solo nos interesa una columna en particular de todo el dataset y ese es la columna de texto, que contiene
 el contenido de los tweets en sí. Para leer el fichero de datos y procesarlo, utilizaremos pandas.

In [1]:
import pandas as pd

Definimos la función que se encarga de leer y procesar el fichero de datos. Esta función eliminará las columnas irrelevantes, seleccionará un subset de 5000 tweets que formará el corpus de nuestros modelos y por último los guardará en otro fichero para su uso más adelante.

In [11]:
def create_cleaned_csv(original_filename, target_filename):
    #uft-8 encoding didn't work, latin1 encoding did.
    data = pd.read_csv(original_filename, encoding="latin1", header=None)

    #We give the columns a name
    column_names = ["target", "id", "date", "flag", "user", "text"]
    data.columns = column_names

    #We eliminate innecessary columns
    data.drop(columns=["target", "id", "date", "flag", "user"], inplace=True)

    #We select a subset of 5000 tweets
    data = data[:5000]

    #Save data onto new file
    data.to_csv(target_filename, index=False)
    print("FINAL RESULT: \n")
    print(data)

In [12]:
create_cleaned_csv("tweet_data.csv", "subset_tweet_data.csv")        

FINAL RESULT: 

                                                   text
0     @switchfoot http://twitpic.com/2y1zl - Awww, t...
1     is upset that he can't update his Facebook by ...
2     @Kenichan I dived many times for the ball. Man...
3       my whole body feels itchy and like its on fire 
4     @nationwideclass no, it's not behaving at all....
...                                                 ...
4995                                    long day today 
4996                     a friend broke his promises.. 
4997               @gjarnling I am fine thanks - tired 
4998          trying to keep my eyes open..damn baking 
4999                        why the hell is it snowing 

[5000 rows x 1 columns]


A continuación, utilizamos la libreria NLTK para limpiar, tokenizar y lematizar nuestro dataset.

Para esto, descargamos los recursos necesarios para trabajar con la librería nltk.

In [23]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\juanq\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

Para asegurarnos de que el recurso que contiene las stopwords se descargó correctamente, los mostramos por pantalla.

In [6]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{'where', 'aren', 'that', "mightn't", "shouldn't", "won't", 'through', 'they', 'until', 'don', 'most', 'which', "wasn't", "you've", 'at', 'who', 'each', 'not', 'having', 'by', 'the', 'both', 'between', 'just', 'd', 'your', 'while', 'too', 'are', 'herself', 'from', 'then', 'under', 'no', 'his', 'any', 're', 'ourselves', 'what', 'some', 'when', 'very', "hadn't", 'did', 'so', 'before', 'we', 'again', 'couldn', 'i', 'do', 'was', 'why', 'nor', 'me', "isn't", 'into', 'now', 'ours', 'he', "mustn't", 'has', "didn't", 'because', "doesn't", 'same', "wouldn't", 's', 'm', 'off', 'but', 'own', 'all', 'as', 'in', 'here', 'about', 'over', 'few', "you'd", 'whom', 'up', 'being', 'during', 'hers', 'shouldn', 'my', 'after', "don't", 'more', 'those', 'below', 'and', 'a', 'these', 'such', 'an', 'll', 'had', 'hadn', 'should', 'haven', 'needn', 'down', 'other', 'above', 'is', 'once', 've', 'or', "needn't", 'were', "aren't", 'our', 'on', 'wasn', "it's", 'does', 'am', 'them', "shan't", 'weren', 'y', 'itself', 

Definimos la función encargada de limpiar los tweets del dataset que eliminará las menciones, hashtags, urls y otros símbolos extraños contenidos en ellos.

Además después de pasar por este filtro, los tokenizamos con TweetTokenizer que a diferencia del tokenizador por defecto de NLTK, word_tokenizer, este si mantiene la coherencia en palabras complejas del inglés.

Por ejemplo la palabra "can't":
- Con word_tokenizer-> can't = ["ca", "n't"]
- Con TweetTokenize-> can't = ["can't"]

In [51]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

def clean_text(original_filename, target_filename):
    data = pd.read_csv(original_filename, encoding="latin1")

    stop_words = set(stopwords.words('english'))

    tt = TweetTokenizer() #Tokenizer
    ps = PorterStemmer() #Stemmer

    for i, row in data.iterrows():
        tweet = data.iloc[i]["text"]
        filtered_tweet = [word for word in tweet.split() if 
                                    word.lower()[0].isalnum() and #Eliminate mentions, hashtags
                                    word.lower() not in stop_words and #Eliminate stop words
                                    'http' not in word.lower()] #Eliminate urls
        
        filtered_tweet_to_string = ' '.join(str(word) for word in filtered_tweet)

        tweet_tokens = tt.tokenize(filtered_tweet_to_string) #Tokenizing

        stemmed_tweet_tokens = [ps.stem(token) for token in tweet_tokens] #Stemming
        data.loc[i,['text']] = ' '.join(str(token) for token in stemmed_tweet_tokens)

    data.to_csv(target_filename, index=False)
    print("FINAL RESULT: \n")
    print(data)

In [52]:
clean_text('subset_tweet_data.csv', 'tokenized_data.csv')

FINAL RESULT: 

                                                   text
0     awww , that' bummer . shoulda got david carr t...
1     upset can't updat facebook text it ... might c...
2     dive mani time ball . manag save 50 % rest go ...
3                       whole bodi feel itchi like fire
4     no , behav all . i'm mad . here ? can't see th...
...                                                 ...
4995                                     long day today
4996                             friend broke promis ..
4997                                    fine thank tire
4998                     tri keep eye open .. damn bake
4999                                          hell snow

[5000 rows x 1 columns]


In [3]:
# Processing data

def probability_word_tagged(word, tag, tweets):

  tweets_tagged_like = {tw for tw in tweets if tw['tag'] == tag}
  n = len(tweets_tagged_like)

  tweets_tagged_like_with_word = {tw for tw in tweets if word in tw['text'].split()}
  m = len(tweets_tagged_like_with_word)

  return m/n

def basic_voting(tweets):

  tag = ''
  i = 0

  tags = [t.get('tag') for t in tweets]

  for t in tuple(tags):
    n = tags.count(t)
    
    if i < n:
      i = n
      tag = t
  
  return tag

In [1]:
# Basic math algorithms

def map_algorithm(candidates):

  max = {'probability': .0}

  for candidate in candidates:
    probability = candidate['probability']

    if(max['probability'] < probability):
      max = candidate

  return max

def basic_distance(tweet1, tweet2):

  distance = 0

  text1 = tweet1('text')
  text2 = tweet2('text')

  for word1 in text1:
    for word2 in text2:
      if word1 != word2:
        distance += 1
  
  return distance

In [2]:
# Algorithms of machine learning

from itertools import islice

def naive_bayes():
    return 0

def basic_kNN(tweet, tweets, k):

  res = {}

  for text, tag in tweets:
    if text != tweet.get('text'):

      v = [tag, basic_distance(tweet, {text: tag})]
      res['text'] = v
  
  res = dict(sorted(res.items(), key=lambda item: item[1][1]))
  iterate_res = iter(res)

  kNN = list(islice(iterate_res, k))

  return basic_voting(kNN)